In [1]:
from EventDriven.backtest import OptionSignalBacktest
import pandas as pd
import numpy as np
from IPython.display import clear_output
from copy import deepcopy


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-02 22:23:29 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
import json
keys = [9, 10, 11, 12, 13, ]
imports_trades = {key: {'trades': pd.read_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_trades_{key}.csv').iloc[:, 1:]} for key in keys}

for key in keys:
    with open(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_weights_{key}.json', 'r') as f:
        weights = json.load(f)
        symbol_list = imports_trades[key]['trades'].Ticker.unique()
        untraded_symbols = [s for s in weights.keys() if s not in imports_trades[key]['trades'].Ticker.unique()]
        for s in untraded_symbols:
            weights.pop(s)
        imports_trades[key]['weights'] = weights

In [35]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
initial_capital = 20_000
capital_tracking = {}
objects = {}
for key in keys:
  print(f"Starting backtest for key: {key}, with initial capital: {initial_capital}")
  trades_ = imports_trades[key]['trades']
  weights = imports_trades[key]['weights']
  objects[key] = {}
  print(weights)
  
  ## Initialize the backtest class
  evb_backtest = OptionSignalBacktest(trades_, initial_capital=initial_capital)

  ## Set the attributes
  w_map = {x: w  * 0.99 for x, w in weights.items()} ## 75% of the weights for each stock
  evb_backtest.portfolio.weight_map = w_map
  evb_backtest.portfolio.weight_map
  evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
  evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
  evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
  evb_backtest.portfolio.order_settings = {'type': 'naked',
  'specifics': [{'direction': 'long',
    'rel_strike': .850,
    'dte': 300,
    'moneyness_width': 0.35},
    {'direction': 'short',
    'rel_strike': .60,
    'dte': 300,
    'moneyness_width': 0.35}
  ],
  'name': 'vertical_spread'}


  evb_backtest.portfolio.max_contract_price = 2
  evb_backtest.executor.commission_rate = 0.65/100
  evb_backtest.executor.commission_rate
  evb_backtest.executor.max_slippage_pct = 0.075
  evb_backtest.portfolio.allocated_cash_map

  ## Run the backtest
  evb_backtest.run()

  ## Get the ending capital
  ending_capital = evb_backtest.portfolio._equity['Total'][-1]
  initial_capital = ending_capital
  capital_tracking[key] = ending_capital
  objects[key]['WEIGHTS'] = w_map
  objects[key]['END_CASH'] = ending_capital
  objects[key]['TEST_RESULT'] = {}
  objects[key]['TEST_RESULT']['BACKTESTER'] = evb_backtest.portfolio
  objects[key]['TEST_RESULT']['AGG'] = evb_backtest.portfolio.aggregate()


  clear_output(wait=True)

Starting backtest for key: 13, with initial capital: 72327.44448288428
{'TSLA': 0.1278137844468317, 'JPM': 0.06492953363063281, 'AAPL': 0.053559853392628316, 'BAC': 0.06081664642539407, 'COST': 0.07417229229710198, 'GOOG': 0.0372945992617209, 'CAT': 0.028522357435578516, 'WMT': 0.045548893698452025, 'BA': 0.01937959395180522, 'DIS': 0.016669716149724515, 'SBUX': 0.016400735830984087, 'HD': 0.021524599489829287}
        Date  AAPL  BA  GOOG  TSLA  DIS  BAC  HD  CAT  JPM  SBUX  COST  WMT
0 2025-01-03     1   1     1     1    1    0   0    0    0     0     0    0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 16 event(s)
    

In [7]:
ending_capital

52149.91237267908

In [19]:
objects[9]

{'WEIGHTS': {'TSLA': 0.09649982487485825,
  'MSFT': 0.07202554177332188,
  'AAPL': 0.059949000853097366,
  'MU': 0.03949630917345526,
  'COST': 0.054014406481527596,
  'TSM': 0.035311767880299195,
  'BA': 0.0330897363848411,
  'HD': 0.033552818549636494,
  'SBUX': 0.02933166053931343,
  'CAT': 0.029306974301299,
  'BAC': 0.02110963276700258,
  'UNG': 0.005711066418292687,
  'GLD': 0.013638894341734648},
 'END_CASH': 17633.814115152483,
 'TEST_RESULT': {'BACKTESTER': <EventDriven.portfolio.OptionSignalPortfolio at 0x13db4da50>,
  'AGG': Start                                                    2022-01-04 00:00:00
  End                                                      2022-12-30 00:00:00
  Duration                                                   360 days 00:00:00
  Exposure Time [%]                                                      53.28
  Equity Final [$]                                                    17633.81
  Equity Peak [$]                                                

In [20]:
imports_trades[9]['trades'].PnL.sum(),imports_trades[9]['trades'].PnL.shape

(-3561.884013580584, (36,))

In [33]:
key = 9
objects[key]['TEST_RESULT']['BACKTESTER'].get_trades().PnL.sum(), imports_trades[key]['trades'].PnL.sum(), imports_trades[key]['trades'].PnL.shape, objects[key]['TEST_RESULT']['BACKTESTER'].get_trades().PnL.shape

(-2366.1858848475167, -3561.884013580584, (36,), (17,))

In [62]:

imports_trades[9]['weights']['NVDA'] * 20000

3488.247915346469